In [21]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
%matplotlib inline 
warnings.filterwarnings("ignore")

from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [22]:
def get_im_cv2(path):
    img = cv2.imread(path)
    height, width, channels = img.shape
    img = cv2.resize(img, (64, 64))
    return img

In [23]:
def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['Automobile',
               'Comics_and_Cartoons',
               'Famous_Personalities',
               'Festivals_and_Occasions',
               'Graffiti_and_Illustrations',
               'Movies_and_TV_shows',
               'Music',
               'Nature',
               'No_Theme',
               'Patterns_and_Ethnic',
               'Signs_and_Symbols',
               'Spiritual',
               'Sports',
               'Superheroes',
               'Typography',
               'Vintage'
              ]
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Train', fld, '*.jpg')
        files = glob.glob(path)
        for i in range(len(files)):
            flbase = os.path.basename(files[i])
            img = get_im_cv2(files[i])
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index) 
    print len(X_train)
    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id

In [24]:
def load_test():
    path = os.path.join('D:\Misc\CrowdAnalytix\Identifying Themes from Mobile Case Images\Data Raw\Test', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id

In [25]:
Xtrall, Ytrall, Itrall = load_train()
X_test, X_test_id = load_test()

Read train images
Load folder Automobile (Index: 0)
Load folder Comics_and_Cartoons (Index: 1)
Load folder Famous_Personalities (Index: 2)
Load folder Festivals_and_Occasions (Index: 3)
Load folder Graffiti_and_Illustrations (Index: 4)
Load folder Movies_and_TV_shows (Index: 5)
Load folder Music (Index: 6)
Load folder Nature (Index: 7)
Load folder No_Theme (Index: 8)
Load folder Patterns_and_Ethnic (Index: 9)
Load folder Signs_and_Symbols (Index: 10)
Load folder Spiritual (Index: 11)
Load folder Sports (Index: 12)
Load folder Superheroes (Index: 13)
Load folder Typography (Index: 14)
Load folder Vintage (Index: 15)
4592
Read train data time: 286.95 seconds


In [35]:
# flatten out all images to be one-dimensional
Xtrall = np.array(Xtrall, dtype=np.uint8)
Ytrall = np.array(Ytrall, dtype=np.uint8)

Xtrall = Xtrall.astype('float32')
Xtrall = Xtrall / 255
Xtrall = Xtrall.transpose((0, 3, 1, 2))

msk = np.random.rand(len(Xtrall)) < 0.8
Xval = Xtrall[~msk]
Yval = Ytrall[~msk]
Ytr = Ytrall[msk]
Xtr = Xtrall[msk]

X_test = np.array(X_test, dtype=np.uint8)

X_test = X_test.astype('float32')
X_test = X_test / 255
X_test = X_test.transpose((0, 3, 1, 2))


In [38]:
Ytr = np_utils.to_categorical(Ytr, 16)
Yval = np_utils.to_categorical(Yval, 16)

In [56]:
batch_size=32
nb_classes=len(Ytr)
nb_epoch=30
nb_filters=32
nb_pool=1
nb_conv=5

In [57]:
model= Sequential()
model.add(Convolution2D(nb_filters,nb_conv,nb_conv, subsample=(1, 1), border_mode='valid',input_shape=Xtr.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters,nb_conv,nb_conv))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool,nb_pool)))
model.add(Dropout(0.5));
model.add(Flatten());
model.add(Dense(128));
model.add(Dropout(0.5));
model.add(Dense(16));
model.add(Activation('softmax'));
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])


In [58]:
print model.output_shape

(None, 16)


In [59]:
model.fit(Xtr,Ytr,batch_size=batch_size,nb_epoch=nb_epoch,verbose=1,validation_data=(Xval, Yval))

Train on 3700 samples, validate on 892 samples
Epoch 1/30
3700/3700 [==============================] - 123s - loss: 2.1993 - acc: 0.4246 - val_loss: 1.5750 - val_acc: 0.5448
Epoch 2/30
3700/3700 [==============================] - 128s - loss: 1.5024 - acc: 0.5759 - val_loss: 1.4369 - val_acc: 0.5942
Epoch 3/30
3700/3700 [==============================] - 131s - loss: 1.2290 - acc: 0.6386 - val_loss: 1.3375 - val_acc: 0.6110
Epoch 4/30
3700/3700 [==============================] - 123s - loss: 0.9613 - acc: 0.7170 - val_loss: 1.3802 - val_acc: 0.6211
Epoch 5/30
3700/3700 [==============================] - 124s - loss: 0.7917 - acc: 0.7730 - val_loss: 1.3490 - val_acc: 0.6401
Epoch 6/30
3700/3700 [==============================] - 124s - loss: 0.6189 - acc: 0.8168 - val_loss: 1.3211 - val_acc: 0.6401
Epoch 7/30
3700/3700 [==============================] - 125s - loss: 0.5132 - acc: 0.8511 - val_loss: 1.4062 - val_acc: 0.6491
Epoch 8/30
3700/3700 [==============================] - 130s - l

In [60]:
batch_size=32
predictions = model.predict(X_test,batch_size, verbose=2)

In [61]:
params = {"0":"Automobile","1":"Comics_and_Cartoons","2":"Famous_Personalities","3":"Festivals_and_Occasions",
              "4":"Graffiti_and_Illustrations","5":"Movies_and_TV_shows","6":"Music","7":"Nature","8":"No_Theme",
              "9":"Patterns_and_Ethnic","10":"Signs_and_Symbols","11":"Spiritual","12":"Sports","13":"Superheroes",
              "14":"Typography","15":"Vintage"}
    
y = np.array([params[str(i)] for i in np.argmax(predictions, axis=1)])

In [62]:
result1 = pd.DataFrame(X_test_id, columns=['id'])
result1.loc[:, 'Mobile_Theme'] = pd.Series(y, index=result1.index)
now = datetime.datetime.now()
sub_file = 'submission_keras_v1' +  '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
result1.to_csv(sub_file, index=False)